# Skip this. Save it for a challenge later. Trying to find all lat and long missing and approximating it

In [199]:
# na_gps_rows = df[(df['GPS_LONGITUDE'].isna()) | (df['GPS_LATITUDE'].isna())]
# vid_list_nans = na_gps_rows['VEHICLE_ID'].drop_duplicates(keep='first').tolist()

In [200]:
# def fill_lat_long(row):
#     if pd.isna(row['GPS_LATITUDE']) or pd.isna(row['GPS_LONGITUDE']):
#         prev_lat = df.at[row.name - 1, 'GPS_LATITUDE']
#         prev_long = df.at[row.name - 1, 'GPS_LONGITUDE']
#         a_distance = df.at[row.name - 1, 'METERS']
#         b_distance = row['METERS']
        
#         filled_lat, filled_long = lat_long_filler((prev_lat, prev_long), a_distance, b_distance)
#         return pd.Series({'GPS_LATITUDE': filled_lat, 'GPS_LONGITUDE': filled_long})
#     else:
#         return row[['GPS_LATITUDE', 'GPS_LONGITUDE']]

In [201]:
# for id in vid_list_nans:
#     cur_vid_rows = df[df['VEHICLE_ID'] == id]
#     if (cur_vid_rows['GPS_LATITUDE'].isna().any()) or (cur_vid_rows['GPS_LONGITUDE'].isna().any()):
#         caught_series = cur_vid_rows.apply(fill_lat_long, axis=1)
#         df[df['VEHICLE_ID'] == id] = caught_series

# Setup

In [202]:
from src.subpipe.validate import ValidateBusData
import logging, os, sys, json
import pandas as pd

from src.utils.utils import (
    DATA_MONTH_DAY,
    SUBSCRIBER_DATA_PATH_JSON,
    SUBSCRIBER_FOLDER,
    curr_time_micro,
    sub_logger,
    lat_long_filler
)

logging.basicConfig(
        format="",
        filename=f"logs/notebook-{DATA_MONTH_DAY}.log",
        encoding="utf-8",
        filemode="a",
        level=logging.INFO,
)

# Code For Transform

In [203]:
df = pd.read_json(os.path.join(SUBSCRIBER_FOLDER, "04-12.json"))
df.size

2953540

In [204]:
df = df.sort_values(["VEHICLE_ID", "ACT_TIME"], ascending=True)
df.size

2953540

In [205]:
df = df.dropna()
df.size

2952180

In [206]:
df = df.reset_index(level=0)
df.size

3247398

In [207]:
df = df.drop(columns=['index', 'GPS_SATELLITES', 'GPS_HDOP'])
df.size

2361744

In [208]:
df.insert(5, 'TIMESTAMP', 0)

In [209]:
date_format = "%d%b%Y:%H:%M:%S"

In [210]:
opd_sec: pd.Timestamp = pd.to_datetime(pd.to_datetime(df['OPD_DATE'], format=date_format), unit='s')

In [211]:
td: pd.Timedelta = pd.to_timedelta(df['ACT_TIME'], unit='sec')

In [212]:
df['TIMESTAMP'] = opd_sec + td
df.drop(columns=['OPD_DATE'], inplace=True)

df.head()


,EVENT_NO_TRIP,EVENT_NO_STOP,VEHICLE_ID,METERS,TIMESTAMP,ACT_TIME,GPS_LONGITUDE,GPS_LATITUDE
0,219000623,219000624,2905,37,2022-12-13 06:18:00,22680,-122.843617,45.504060
1,219000623,219000624,2905,47,2022-12-13 06:18:05,22685,-122.843743,45.504095
2,219000623,219000624,2905,54,2022-12-13 06:18:10,22690,-122.843828,45.504118
3,219000623,219000626,2905,181,2022-12-13 06:19:00,22740,-122.843408,45.505170
4,219000623,219000626,2905,196,2022-12-13 06:19:05,22745,-122.843285,45.505283


In [213]:
df.insert(4, 'SPEED', 0)
df.insert(4, 'dMETERS', 0)
df.insert(4, 'dTIMESTAMP', 0)

In [214]:
df.head()

,EVENT_NO_TRIP,EVENT_NO_STOP,VEHICLE_ID,METERS,dTIMESTAMP,dMETERS,SPEED,TIMESTAMP,ACT_TIME,GPS_LONGITUDE,GPS_LATITUDE
0,219000623,219000624,2905,37,0,0,0,2022-12-13 06:18:00,22680,-122.843617,45.504060
1,219000623,219000624,2905,47,0,0,0,2022-12-13 06:18:05,22685,-122.843743,45.504095
2,219000623,219000624,2905,54,0,0,0,2022-12-13 06:18:10,22690,-122.843828,45.504118
3,219000623,219000626,2905,181,0,0,0,2022-12-13 06:19:00,22740,-122.843408,45.505170
4,219000623,219000626,2905,196,0,0,0,2022-12-13 06:19:05,22745,-122.843285,45.505283


In [224]:
vid_list = df['VEHICLE_ID'].drop_duplicates(keep='first').tolist()

for id in vid_list:
    curr_data = df[df['VEHICLE_ID'] == id]
    curr_data.loc[0, 'SPEED'] = curr_data.loc[0, 'METERS']

KeyError: 0